# Mycologue Labs - Image to 3D workflow

The following workflow demonstrates how to turn a folder of images of a mushroom specimen into a high fidelity 3D model.

This workflow does not include the processing of RAW images to color corrected JPG images, which is highly recommended for quality 3D model textures.

In [ ]:
# define the location of your image input directory
project_name = '20241003-1549-russula-brevipes'
input_dir = f'photos/{project_name}'

In [10]:
!pwsh -Command "& './create-masks.ps1' '{input_dir}'"

Skipping C:\Users\ucodia\mycologue\photos\20241011-1408-admirable-bolete\DSC04131.jpg (mask exists)
Skipping C:\Users\ucodia\mycologue\photos\20241011-1408-admirable-bolete\DSC04127.jpg (mask exists)
Skipping C:\Users\ucodia\mycologue\photos\20241011-1408-admirable-bolete\DSC04130.jpg (mask exists)
Skipping C:\Users\ucodia\mycologue\photos\20241011-1408-admirable-bolete\DSC04129.jpg (mask exists)
Skipping C:\Users\ucodia\mycologue\photos\20241011-1408-admirable-bolete\DSC04128.jpg (mask exists)
Skipping C:\Users\ucodia\mycologue\photos\20241011-1408-admirable-bolete\DSC04132.jpg (mask exists)
Skipping C:\Users\ucodia\mycologue\photos\20241011-1408-admirable-bolete\DSC04133.jpg (mask exists)
Skipping C:\Users\ucodia\mycologue\photos\20241011-1408-admirable-bolete\DSC04135.jpg (mask exists)
Skipping C:\Users\ucodia\mycologue\photos\20241011-1408-admirable-bolete\DSC04134.jpg (mask exists)
Skipping C:\Users\ucodia\mycologue\photos\20241011-1408-admirable-bolete\DSC04136.jpg (mask exists)
